In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv(r"SBS_Processed_Datasets/02_cleaned_output.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8189 entries, 0 to 8188
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             8189 non-null   object 
 1   transmission     8189 non-null   object 
 2   owner_no         8189 non-null   int64  
 3   brand            8189 non-null   object 
 4   model            8189 non-null   object 
 5   model_year       8189 non-null   int64  
 6   variant_name     8189 non-null   object 
 7   price            8189 non-null   float64
 8   registered_year  8189 non-null   int64  
 9   fuel_type        8189 non-null   object 
 10  kms_driven       8189 non-null   int64  
 11  engine_cc        8189 non-null   int64  
 12  mileage_kmpl     8189 non-null   float64
dtypes: float64(2), int64(5), object(6)
memory usage: 831.8+ KB


In [4]:
df.head()

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,kms_driven,engine_cc,mileage_kmpl
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,120000,998,23.10
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,32706,1497,17.00
2,bangalore,manual,1,tata,tata tiago,2018,1.2 revotron xz,585000.0,2018,petrol,11949,1199,23.84
3,bangalore,manual,1,hyundai,hyundai xcent,2014,1.2 kappa s option,462000.0,2014,petrol,17794,1197,19.10
4,bangalore,manual,1,maruti,maruti sx4 s cross,2015,ddis 200 zeta,790000.0,2015,diesel,60000,1248,23.65


FEATURE ENGINEERING

In [5]:
#STEP_1 ADD AGE COLUMN

# Get the current year
current_year = datetime.now().year

# Calculate car age
df['car_age'] = current_year - df['registered_year']

In [6]:
#STEP_2 Create mileage_normalized

# Calculate normalized mileage
df['mileage_normalized'] = df['mileage_kmpl'] / df['car_age']

In [7]:
#STEP_3 brand_popularity

# Calculate brand popularity as average price per brand
brand_popularity = df.groupby('brand')['price'].mean().to_dict()
df['brand_popularity'] = df['brand'].map(brand_popularity)

In [8]:
#STEP_5 Age Bins

# Define age bins and labels
age_bins = [0, 2, 7, float('inf')]
age_labels = ['new', 'mid-age', 'old']
df['age_group'] = pd.cut(df['car_age'], bins=age_bins, labels=age_labels)

In [9]:
#STEP_6 High Mileage Flag

# Flag for cars with high mileage
df['high_mileage'] = (df['mileage_kmpl'] > df['mileage_kmpl'].quantile(0.75)).astype(int)

In [10]:
df.head()

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,kms_driven,engine_cc,mileage_kmpl,car_age,mileage_normalized,brand_popularity,age_group,high_mileage
0,bangalore,manual,3,maruti,maruti celerio,2015,vxi,400000.0,2015,petrol,120000,998,23.10,9,2.566667,504649.260948,old,1
1,bangalore,manual,2,ford,ford ecosport,2018,1.5 petrol titanium bsiv,811000.0,2018,petrol,32706,1497,17.00,6,2.833333,730169.934641,mid-age,0
2,bangalore,manual,1,tata,tata tiago,2018,1.2 revotron xz,585000.0,2018,petrol,11949,1199,23.84,6,3.973333,748643.178049,mid-age,1
3,bangalore,manual,1,hyundai,hyundai xcent,2014,1.2 kappa s option,462000.0,2014,petrol,17794,1197,19.10,10,1.910000,605200.000615,old,0
4,bangalore,manual,1,maruti,maruti sx4 s cross,2015,ddis 200 zeta,790000.0,2015,diesel,60000,1248,23.65,9,2.627778,504649.260948,old,1


In [11]:
df.describe()

,owner_no,model_year,price,registered_year,kms_driven,engine_cc,mileage_kmpl,car_age,mileage_normalized,brand_popularity,high_mileage
count,8189.000000,8189.000000,8.189000e+03,8189.000000,8.189000e+03,8189.000000,8189.000000,8189.000000,8189.000000,8.189000e+03,8189.000000
mean,1.362560,2016.492246,9.085296e+05,2016.559409,5.921017e+04,1426.188668,19.202011,7.440591,3.634105,9.085296e+05,0.248016
std,0.633744,3.879895,1.045562e+06,3.888378,7.465923e+04,468.752493,4.037720,3.888378,2.822979,7.846038e+05,0.431887
min,1.000000,1985.000000,2.800000e+04,1985.000000,1.010000e+02,72.000000,7.080000,1.000000,0.328205,7.333333e+04,0.000000
25%,1.000000,2014.000000,3.990000e+05,2014.000000,3.053800e+04,1197.000000,17.000000,5.000000,1.933333,5.046493e+05,0.000000
50%,1.000000,2017.000000,6.000000e+05,2017.000000,5.400000e+04,1248.000000,18.900000,7.000000,2.833333,6.052000e+05,0.000000
75%,2.000000,2019.000000,9.490000e+05,2019.000000,8.000000e+04,1498.000000,21.400000,10.000000,4.280000,7.486432e+05,0.000000
max,5.000000,2023.000000,9.600000e+06,2023.000000,5.500000e+06,5000.000000,140.000000,39.000000,35.600000,6.343750e+06,1.000000


In [12]:
df[df["engine_cc"] < 100]

,city,transmission,owner_no,brand,model,model_year,variant_name,price,registered_year,fuel_type,kms_driven,engine_cc,mileage_kmpl,car_age,mileage_normalized,brand_popularity,age_group,high_mileage
562,bangalore,automatic,1,mahindra,mahindra e2o plus,2017,p6,550000.0,2017,electric,20000,72,110.0,7,15.714286,930921.126761,mid-age,1
817,bangalore,automatic,2,mahindra,mahindra e2o plus,2017,p8,460000.0,2017,electric,20000,72,140.0,7,20.000000,930921.126761,mid-age,1


In [13]:
# Export it
df.to_csv(r"SBS_Processed_Datasets\03_feature_engineered_output.csv",index=False)